# 연금 (pension)
- 소득이 줄어드는 노후에 대비해 미리 자금을 적립해 두고, 이후 일정 시점부터 정기적으로 지급받는 현금흐름

### 연금의 3층탑
![연금의 3층탑](https://cdn.sisajournal-e.com/news/photo/202309/303760_128015_2849.png)


### 국민연금
- 가입대상 : 18세 이상 60세 미만 대한민국 국민
- 가입유형

| 가입유형    | 대상                   | 의무/임의 | 보험료율                    |
|---------|----------------------|-------|-------------------------|
| 직장가입자   | 회사에 다니는 사람           | 의무    | 9% (사업주 4.5%, 근로자 4.5%) |
| 지역가입자   | 자영업자, 프리랜서 등         | 의무    | 9%                      |
| 임의가입자   | 18세 이상 60세 미만 국민 누구나 |       | 임의                      |
| 임의계속가입자 | 60세 이후에도 계속 가입 가능    |       | 임의                      |


- 급여종류 : 노령연금, 장애연금, 유족연금

| 급여종류 | 수급요건                  | 지급대상   | 지급액 산정기준                                                 |
|------|-----------------------|--------|----------------------------------------------------------|
| 노령연금 | 가입기간 10년 이상, 만 65세 이상 | 가입자 본인 | 가입기간, 가입자 평균소득월액                                         |
| 장애연금 | 가입기간 1년 이상, 장애등급 1~6급 | 가입자 본인 | 가입기간, 가입자 평균소득월액, 장애등급                                   |
| 유족연금 | 가입기간 1년 이상, 가입자 사망    | 유족     | 기존 수령액의 40% - 60% (가입기간에 따른차등 10년 40%, 20년 50%, 30년 60%) |

> 급여액 산정 : 가입기간, 가입자 평균소득월액 기준으로 산정됨
> [노령연금 예상연금월액표](https://www.nps.or.kr/pnsinfo/databbs/getOHAF0272M1Detail.do?menuId=MN24001000&pstId=ZZ202500000000000352&hmpgCd=&hmpgBbsCd=BS20240094&sortSe=FR&pageIndex=1&searchText=&searchGbu=)

- 조기 지연 수령
    - 조기수령 : 1년 단위로 6% 감액 (최대 5년 30% 감액)
    - 지연수령 : 1년 단위로 7.2% 가산 (최대 5년 36% 증액)
    - *조기, 지연 수령의 장단점 확인 필요*


#### 국민연금의 이익 회수
- 국민연금은 공적부조 제도로 내는만큼 받는 제도가 아님
- 적게내는 사람은 많이 받고 많이 내는 사람은 적게 받음

| 가입기간(월) | 월소득       | 연금보험료   | 총납입액        | 예상연금월액    | 원금회수기간(월) |
|---------|-----------|---------|-------------|-----------|-----------|
| 120     | 400,000   | 36,000  | 4,320,000   | 175,760   | 25        |
| 240     | 5,000,000 | 450,000 | 108,000,000 | 811,930   | 134       |
| 360     | 6,370,000 | 573,300 | 206,388,000 | 1,422,400 | 146       |


무소득자도 소액투자로 최소한의 연금수령권 확보 필요
- 추가납부 활용
- 임의 계속 가입자 활용


In [9]:
%use kandy
%use dataframe

In [126]:
class Item(
    val years: Int,
    val monthly_payment: Double,
    val rest_years: Int,
    val pension_years: Int,
    val monthly_pension_amount: Double,
    val self_employed: Boolean = true,
)

fun nps(
    item : Item,
    interest: Double
): List<Double> {
    val monthly_payment = item.monthly_payment
    val monthly_pension_amount = item.monthly_pension_amount
    val months = item.years * 12
    val rest_months = item.rest_years * 12
    val pension_months = item.pension_years * 12
    val self_employed = item.self_employed

    val list = List(months + rest_months + pension_months) {
        when {
            it < months -> {
                val monthly_payment1 = when {
                    it < months / 5.0 -> monthly_payment * 0.5
                    it < months / 5.0 * 2 -> monthly_payment * 0.75
                    it < months / 5.0 * 3 -> monthly_payment
                    it < months / 5.0 * 4 -> monthly_payment * 1.25
                    it < months / 5.0 * 5 -> monthly_payment * 1.5
                    else -> monthly_payment
                }

                if (self_employed) {
                    -monthly_payment1
                } else {
                    -monthly_payment1 / 2
                }
            }
            it < months + rest_months -> {
                0.0
            }
            else -> {
                monthly_pension_amount
            }
        }
    }.runningFold(0.0) { acc, i ->
        (acc + i) * (1 + interest / 12)
    }.drop(1)

    return list
}


In [141]:
val years = 25
val pension_years = 30
val interest_rate = 0.00

// 30살 취업 + 55살 퇴직
// 25년 일 후 65살부터 연금수령
// 조기수령시 60, 지연수령시 70

val items = listOf(
    Item(years, 40 * 0.09, 10, pension_years, 40.0000, true),
    Item(years, 308 * 0.09, 10, pension_years, 77.3440, false),
    Item(years, 500 * 0.09, 10, pension_years, 101.4160, false),
//    Item(years, 500 * 0.09, 5, pension_years + 5, 101.4160 * 0.7, false),
//    Item(years, 500 * 0.09, 15, pension_years - 5, 101.4160 * 1.36, false),
//    Item(years, 500 * 0.09, 10, pension_years, 101.4160, true),

)

val df = dataFrameOf(
    "m" to List((years + 10 + pension_years) * 12) { it + 1 },
    *items.mapIndexed { index, item ->
        "item$index" to nps(
            item = item,
            interest = interest_rate,
        )
    }.toTypedArray()
)

df.plot {
    items.forEachIndexed { index, item ->
        path {
            x("m") {
                axis.breaksLabeled(
                    *List(50) { it * 60 to "${(it * 5) + 30}" }.toTypedArray()
                )
            }
            y("item$index")
        }
    }
    layout {
        size = 1200 to 600
    }
}


<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="KJBH0a"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"mapping":{
},
"data":{
"item0":[-1.7999999999999998,-3.5999999999999996,-5.3999999999999995,-7.199999999999999,-9.0,-10.8,-12.600000000000001,-14.400000000000002,-16.200000000000003,-18.000000000000004,-19.800000000000004,-21.600000000000005,-23.400000000000006,-25.200000000000006,-27.000000000000007,-28.800000000000008,-30.60000000000001,-32.400000000000006,-34.2,-36.0,-37.8,-39.599999999999994,-41.39999999999999,-43.19999999999999,-44.999999999999986,-46.79999999999998,-48.59999999999998,-50.39999999999998,-52.199999999999974,-53.99999999999997,-55.79999999999997,-57.599999999999966,-59.39999999999996,-61.19999999999996,-62.99999999999996,-64.79999999999995,-66.59999999999995,-68.39999999999995,-70.19999999999995,-71.99999999999994,-73.79999999999994,-75.59999999999994,-77.39999999999993,-79.19999999999993,-80.99999999999993,-82.79999999999993,-84.59999999999992,-86.39999999999992,-88.19999999999992,-89.99999999999991,-91.79999999999991,-93.59999999999991,-95.3999999999999,-97.1999999999999,-98.9999999999999,-100.7999999999999,-102.5999999999999,-104.39999999999989,-106.19999999999989,-107.99999999999989,-110.69999999999989,-113.39999999999989,-116.0999999999999,-118.7999999999999,-121.4999999999999,-124.1999999999999,-126.8999999999999,-129.5999999999999,-132.2999999999999,-134.9999999999999,-137.69999999999987,-140.39999999999986,-143.09999999999985,-145.79999999999984,-148.49999999999983,-151.19999999999982,-153.8999999999998,-156.5999999999998,-159.29999999999978,-161.99999999999977,-164.69999999999976,-167.39999999999975,-170.09999999999974,-172.79999999999973,-175.49999999999972,-178.1999999999997,-180.8999999999997,-183.59999999999968,-186.29999999999967,-188.99999999999966,-191.69999999999965,-194.39999999999964,-197.09999999999962,-199.7999999999996,-202.4999999999996,-205.1999999999996,-207.89999999999958,-210.59999999999957,-213.29999999999956,-215.99999999999955,-218.69999999999953,-221.39999999999952,-224.0999999999995,-226.7999999999995,-229.4999999999995,-232.19999999999948,-234.89999999999947,-237.59999999999945,-240.29999999999944,-242.99999999999943,-245.69999999999942,-248.3999999999994,-251.0999999999994,-253.7999999999994,-256.4999999999994,-259.19999999999936,-261.89999999999935,-264.59999999999934,-267.29999999999933,-269.9999999999993,-273.59999999999934,-277.19999999999936,-280.7999999999994,-284.3999999999994,-287.99999999999943,-291.59999999999945,-295.1999999999995,-298.7999999999995,-302.3999999999995,-305.99999999999955,-309.59999999999957,-313.1999999999996,-316.7999999999996,-320.39999999999964,-323.99999999999966,-327.5999999999997,-331.1999999999997,-334.7999999999997,-338.39999999999975,-341.9999999999998,-345.5999999999998,-349.1999999999998,-352.79999999999984,-356.39999999999986,-359.9999999999999,-363.5999999999999,-367.19999999999993,-370.79999999999995,-374.4,-378.0,-381.6,-385.20000000000005,-388.80000000000007,-392.4000000000001,-396.0000000000001,-399.60000000000014,-403.20000000000016,-406.8000000000002,-410.4000000000002,-414.0000000000002,-417.60000000000025,-421.2000000000003,-424.8000000000003,-428.4000000000003,-432.00000000000034,-435.60000000000036,-439.2000000000004,-442.8000000000004,-446.40000000000043,-450.00000000000045,-453.6000000000005,-457.2000000000005,-460.8000000000005,-464.40000000000055,-468.00000000000057,-471.6000000000006,-475.2000000000006,-478.80000000000064,-482.40000000000066,-486.0000000000007,-490.5000000000007,-495.0000000000007,-499.5000000000007,-504.0000000000007,-508.5000000000007,-513.0000000000007,-517.5000000000007,-522.0000000000007,

누적 수익률 추이 (기금 설치 후)
- 1988년~2024년 말: 6.82%

기대수명
- 2023년 남성 80.6년, 여성 86.4년



### 퇴직연금

연금 전환시

연금수령한도
$\frac{연금계좌 평가액}{11 - 연금수령 연차} \times 120\%$

퇴직연금은 수령한도 내에서 찾을시에 30% 감면

[연금인출순서](https://www.pwc.com/kr/ko/insights/issue-brief/retirement-pension4.png)


연금 = 국민연금 + 퇴직연금(IRP) + 개인연금

### 세액공제
![](https://mybiz.pay.naver.com/api/files/202402/8-2-1926172304.png)
### 과세이연
- 국민연금 : 납부시 전액 소득공제
- 연금계좌에서 발생한 소득에 대해 즉시 세금을 부과하지 않고, 연금 개시 후 연금액에 따라 세금 부과
### 저율과세
- 국민연금 : 원천징수 (종합과세)
- 퇴직연금 : 퇴직소득세의 70% ~ 60%
- 개인연금 : 5.5% - 3.3% 저율과세


In [5]:
import java.text.NumberFormat
import java.util.Locale

fun 공제율(급여 : Int): Double {
    return if (급여 <= 5500_0000) 0.15 else 0.12
}

val 납입액 = 900_0000
val 급여 = 1_0000_0000

val 세액공제 = 납입액 * 공제율(급여)

val formatter = NumberFormat.getCurrencyInstance(Locale.KOREA)

println("연금 납입액: ${formatter.format(납입액)}")
println("세액공제: ${formatter.format(세액공제)}")


연금 납입액: ₩9,000,000
세액공제: ₩1,080,000


In [124]:
val avg = 500
val years = 25

val total = 500 * 25 * 12
val list = listOf(
    avg * 60 * 0.5,
            avg * 60 * 0.7,
            avg * 60 * 1.0,
            avg * 60 * 1.3,
            avg * 60 * 1.5,
).sum()


println(total)
println(list)



150000
150000.0
